# Construction d'un Arbre de Décision Manuel

Dans ce notebook, nous allons construire manuellement un arbre de décision pour classer les fleurs Iris en fonction de leurs caractéristiques.



## Importation des bibliothèques nécessaires

In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from pprint import pprint
# numpy (np) : Bibliothèque pour les opérations mathématiques efficaces.
# pandas (pd) : Utilisé pour manipuler et analyser les données tabulaires.
# shuffle de sklearn.utils : Utilisé pour mélanger les données.
# pprint : Pretty Print, utilisé pour afficher les structures de données de manière plus lisible.


# Chargement et préparation des données

In [2]:
df = pd.read_csv('data\iris.csv')
df_shuffled = shuffle(df, random_state=200)
train = df_shuffled.sample(frac=0.8, random_state=200)
test = df_shuffled.drop(train.index)
# read_csv de pandas : Utilisé pour lire les données à partir d'un fichier CSV.
# shuffle : Utilisé pour mélanger les données, en particulier lors de l'entraînement des modèles.

#Fonctions pour la Construction de l'Arbre


1.   calculate_best_threshold : Fonction pour déterminer le meilleur seuil de division d'un nœud en fonction du critère Gini.
2.   creation_of_a_node : Fonction pour créer un nœud avec des informations spécifiques.
3.   creation_of_a_conditional_node : Fonction pour créer un nœud conditionnel en fonction du meilleur seuil.
4.   creation_of_the_tree : Fonction principale pour créer l'arbre de décision de manière récursive jusqu'à une certaine profondeur.






In [3]:
def calculate_best_threshold(data, target):
    # Sélection des caractéristiques (colonnes) à partir des données
    features = data.loc[:, data.columns != target]

    # Dictionnaire pour stocker les seuils possibles pour chaque caractéristique
    thresholds = {}

    # Initialisation des meilleures valeurs
    best_threshold = None
    best_gini = float('inf')  # Initialisé à l'infini pour garantir une mise à jour lors de la première itération
    best_feature = None
    best_left_population = None
    best_right_population = None
    best_left_gini = None
    best_right_gini = None
    best_left_node = pd.DataFrame()
    best_right_node = pd.DataFrame()

    # Boucle à travers chaque caractéristique
    for feature in features:
        # Tri des valeurs de la caractéristique pour obtenir les seuils possibles
        sorted_values = data[feature].sort_values().values
        thresholds[feature] = (sorted_values[:-1] + sorted_values[1:]) / 2

        # Boucle à travers chaque seuil
        for threshold in thresholds[feature]:
            # Séparation des données en nœuds gauche et droit selon le seuil
            left_node = data[data[feature] > threshold]
            right_node = data[data[feature] <= threshold]

            # Calcul des populations et des Gini pour les nœuds gauche et droit
            left_population = left_node[target].value_counts() / len(left_node)
            right_population = right_node[target].value_counts() / len(right_node)

            left_gini = 1 - (left_population**2).sum()
            right_gini = 1 - (right_population**2).sum()

            # Calcul du Gini total en combinant les Gini des nœuds gauche et droit
            total_gini = left_gini * len(left_node) / len(data) + right_gini * len(right_node) / len(data)

            # Mise à jour des meilleures valeurs si le Gini total est plus bas
            if total_gini < best_gini:
                best_gini = total_gini
                best_threshold = threshold
                best_feature = feature
                best_left_population = dict(left_population)
                best_right_population = dict(right_population)
                best_left_gini = left_gini
                best_right_gini = right_gini
                best_left_node = left_node.copy()
                best_right_node = right_node.copy()

    # Retourne un dictionnaire contenant les meilleures valeurs trouvées
    return {
        'best_threshold': best_threshold,
        'best_gini': best_gini,
        'best_feature': best_feature,
        'left_population': best_left_population,
        'right_population': best_right_population,
        'left_gini': best_left_gini,
        'right_gini': best_right_gini,
        'left_node': best_left_node,
        'right_node': best_right_node,
    }


In [4]:
def creation_of_a_node(left_population, left_gini, right_population, right_gini):
    node = {
        'Profondeur': 0,
        'Populations de gauche': left_population,
        'Etat noeud gauche': 'Leaf node 🌿' if left_population is not None and len(left_population) == 1 else 'Simple node 🪢',
        'Gini de gauche': left_gini,
        'Populations de droite': right_population,
        'Etat noeud droit': 'Leaf node 🌿' if right_population is not None and len(right_population) == 1 else 'Simple node 🪢',
        'Gini de droite': right_gini,
        'Prédiction gauche': max(left_population, key=left_population.get) if left_population is not None else None,
        'Prédiction droite': max(right_population, key=right_population.get) if right_population is not None else None
    }
    return node


In [5]:
def creation_of_a_conditional_node(data, target, best_feature, best_threshold):
    population = dict(data[target].value_counts() / len(data[target]))
    conditional_node = {
        'Profondeur': 0,
        'Populations': population,
        'Condition': f'{best_feature} > {best_threshold}'
    }
    return conditional_node

In [6]:
def creation_of_the_tree(depth, data, target, tree=None, current_depth=0):
    # Initialisation de l'arbre s'il est None
    if tree is None:
        tree = []

    # Condition d'arrêt : profondeur atteinte
    if depth == 0:
        return tree

    # Calcul des informations sur le meilleur seuil
    info = calculate_best_threshold(data, target)

    # Vérifier la condition indésirable
    if (info['left_gini'] == 0 and info['right_gini'] == 1) or (info['left_gini'] == 1 and info['right_gini'] == 0):
        return tree  # Éviter de construire des nœuds supplémentaires

    if not (info['left_gini'] == 0 and info['right_gini'] == 0) and info['left_gini'] is not None:
        # Création du nœud conditionnel
        conditional_node = creation_of_a_conditional_node(data=data,
                                                          target=target,
                                                          best_feature=info['best_feature'],
                                                          best_threshold=info['best_threshold'])
        if conditional_node not in tree:
            tree.append(conditional_node)
            conditional_node['Profondeur'] = current_depth

            # Création du nœud
            node = creation_of_a_node(left_population=info['left_population'],
                                      left_gini=info['left_gini'],
                                      right_population=info['right_population'],
                                      right_gini=info['right_gini'])
            if node not in tree:
                tree.append(node)
                node['Profondeur'] = current_depth + 1  # incrémenter pour les nœuds enfants

    # Appels récursifs pour les nœuds gauche et droit avec la profondeur mise à jour
    tree = creation_of_the_tree(depth=depth - 1, data=info['left_node'], target=target, tree=tree,
                                current_depth=current_depth + 1)
    tree = creation_of_the_tree(depth=depth - 1, data=info['right_node'], target=target, tree=tree,
                                current_depth=current_depth + 1)

    return tree

# Appel de la fonction
result_tree = creation_of_the_tree(depth=5, data=train, target='variety')
pprint(result_tree, width=200, indent=4)


[   {'Condition': 'petal.length > 1.9', 'Populations': {'Setosa': 0.325, 'Versicolor': 0.35833333333333334, 'Virginica': 0.31666666666666665}, 'Profondeur': 0},
    {   'Etat noeud droit': 'Leaf node 🌿',
        'Etat noeud gauche': 'Simple node 🪢',
        'Gini de droite': 0.0,
        'Gini de gauche': 0.4980948026215516,
        'Populations de droite': {'Setosa': 1.0},
        'Populations de gauche': {'Versicolor': 0.5308641975308642, 'Virginica': 0.4691358024691358},
        'Profondeur': 1,
        'Prédiction droite': 'Setosa',
        'Prédiction gauche': 'Versicolor'},
    {'Condition': 'petal.length > 4.9', 'Populations': {'Versicolor': 0.5308641975308642, 'Virginica': 0.4691358024691358}, 'Profondeur': 1},
    {   'Etat noeud droit': 'Simple node 🪢',
        'Etat noeud gauche': 'Simple node 🪢',
        'Gini de droite': 0.15879017013232533,
        'Gini de gauche': 0.05551020408163265,
        'Populations de droite': {'Versicolor': 0.9130434782608695, 'Virginica': 0.086